<a href="https://colab.research.google.com/github/AtSourav/AE-w.-Bottleneck-Residual-Blocks/blob/main/AE_cifar10_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import math
import tensorflow_datasets as tfds
from keras import losses
from keras import layers
from keras import utils
from keras import metrics
from keras import backend as K
from keras import initializers

import os
import random
import matplotlib.pyplot as plt

In [2]:
! git clone https://github.com/AtSourav/AE-w.-Bottleneck-Residual-Blocks

fatal: destination path 'AE-w.-Bottleneck-Residual-Blocks' already exists and is not an empty directory.


In [3]:
%cd "/content/AE-w.-Bottleneck-Residual-Blocks"

/content/AE-w.-Bottleneck-Residual-Blocks


In [4]:
import plotting_tools as plts
import Bottleneck_residual_blocks as resblock

In [5]:
img_ht = 96
img_wd = 96

input_size = (img_ht,img_wd,3)
latent_dim = 256
batch_size = 128

initializer = initializers.HeNormal(seed=100)

In [6]:
# we're using a combination of max and min pooling, min pooling in the skip connections in the residual blocks,
# max pooling otherwise

encoder_input = keras.Input(shape=input_size)

x = resblock.conv2d_block(64, 3, initializer)(encoder_input)

x = resblock.conv2d_block(64, 3, initializer)(x)

# ------------------------------------------------------------------

x = resblock.conv2d_block(128, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(128, 2, 3, initializer, 'min')(x)
        # the second argument is the compress-ratio for the bottleneck, the third argument is the kernel_size

x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="valid")(x)

# ------------------------------------------------------------------

x = resblock.conv2d_block(256, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(256, 2, 3, initializer, 'min')(x)

x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="valid")(x)

# -------------------------------------------------------------------

x = resblock.conv2d_block(512, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(512, 2, 3, initializer, 'min')(x)

x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="valid")(x)

# -------------------------------------------------------------------

x = resblock.conv2d_block(1024, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(1024, 2, 3, initializer, 'min')(x)

x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="valid")(x)

# -------------------------------------------------------------------

#x = resblock.conv2d_block(2048, 2, initializer)(x)

#x = resblock.bottleneck_residual_conv2D_block(2048, 2, 2, initializer, 'min', padding='same')(x)

# x = layers.MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding="same")(x)

# -------------------------------------------------------------------

x = layers.Flatten()(x)

x = layers.Dense(2*latent_dim)(x)
x = layers.ReLU()(x)

z = layers.Dense(latent_dim, name="z")(x)

encoder = keras.Model(encoder_input, z, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_block (conv2d_block  (None, 94, 94, 64)        2048      
 )                                                               
                                                                 
 conv2d_block_1 (conv2d_blo  (None, 92, 92, 64)        37184     
 ck)                                                             
                                                                 
 conv2d_block_2 (conv2d_blo  (None, 90, 90, 128)       74368     
 ck)                                                             
                                                                 
 bottleneck_residual_conv2d  (None, 88, 88, 128)       54016     
 _block (bottleneck_residua                                

In [8]:
latent_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(2*latent_dim)(latent_input)
x = layers.ReLU()(x)

x = layers.Dense(4*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*2*1024)(x)
x = layers.ReLU()(x)

x = layers.Reshape((2,2,1024))(x)

# ---------------------------------------------------

#x = resblock.conv2dtrans_block(1024, 2, initializer, strides=(2,2))(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(1024, 2, 3, initializer)(x)

# ---------------------------------------------------

x = resblock.conv2dtrans_block(512, 3, initializer, strides=(2,2))(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(512, 2, 3, initializer)(x)

# ---------------------------------------------------

x = resblock.conv2dtrans_block(512, 3, initializer, strides=(2,2))(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(512, 2, 3, initializer)(x)

# ---------------------------------------------------

x = resblock.conv2dtrans_block(256, 3, initializer, strides=(2,2))(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(256, 2, 3, initializer)(x)

x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)

# ---------------------------------------------------

x = resblock.bottleneck_residual_conv2D_block(256, 2, 3, initializer, 'min')(x)

x = resblock.bottleneck_residual_conv2D_block(256, 2, 3, initializer, 'min')(x)

# ---------------------------------------------------

x = resblock.conv2d_block(128, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(128, 2, 3, initializer, 'min')(x)

# ----------------------------------------------------

x = resblock.conv2d_block(64, 2, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(64, 2, 2, initializer, 'min')(x)

# -----------------------------------------------------

decoder_output = layers.Conv2D(3, 1, activation='sigmoid', padding='valid', kernel_initializer=initializer)(x)

decoder = keras.Model(latent_input, decoder_output, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256)]             0         
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 re_lu_51 (ReLU)             (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 1024)              525312    
                                                                 
 re_lu_52 (ReLU)             (None, 1024)              0         
                                                                 
 dense_6 (Dense)             (None, 4096)              4198400   
                                                                 
 re_lu_53 (ReLU)             (None, 4096)              0   

In [9]:
decoder_out = decoder(encoder(encoder_input))
AE = keras.Model(encoder_input, decoder_out, name='AE')

AE.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 encoder (Functional)        (None, 256)               11509440  
                                                                 
 decoder (Functional)        (None, 96, 96, 3)         19278339  
                                                                 
Total params: 30787779 (117.45 MB)
Trainable params: 30770883 (117.38 MB)
Non-trainable params: 16896 (66.00 KB)
_________________________________________________________________


In [10]:
ds = tfds.load('stl10', split='unlabelled[:40%]')

In [11]:
ds_to_np = list(ds.as_numpy_iterator())

In [12]:
random.seed(100)
random.shuffle(ds_to_np)
img_train = np.array([x['image'] for x in ds_to_np[:30000]])
img_valid = np.array([x['image'] for x in ds_to_np[30000:]])

img_train = img_train/255
img_valid = img_valid/255

In [36]:
K.clear_session()

In [13]:
optim = keras.optimizers.Adam(learning_rate = 0.001)
AE.compile(optimizer=optim,loss='mse')
history = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=10, steps_per_epoch=None)

Epoch 1/10


ResourceExhaustedError: ignored